In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, sample_each, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load config & data 

In [6]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [7]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [8]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [9]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [10]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [11]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [ ]:
while epoch < config.n_epochs:
    time_start = time.time()
    sample_each(train_docs, topic_root, train=True)
    time_log = float(time.time() - time_start)
    
    sample_each(dev_docs, topic_root, train=False)
    sample_each(test_docs, topic_root, train=False)
    
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train=0
    ppl_dev = get_perplexity(dev_docs, topic_root)
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, ppl_dev, ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN:      VALID:       TEST: SPEC:             HIER:  \
         Time  Ep Ct    PPL         PPL         PPL     1     2     3 CHILD   
0   65.552253   0  0      0  458.408252  455.688634  0.05  0.42  0.51  0.87   
1   65.076435   1  0      0  451.548189  449.127654  0.05  0.46  0.53  0.82   
2   67.445332   2  0      0  434.564848  430.837731  0.05  0.50  0.55  0.75   
3   68.730501   3  0      0  413.798692  408.206142  0.05  0.52  0.58  0.69   
4   69.845252   4  0      0  389.428408  381.532347  0.05  0.50  0.58  0.59   
5   74.783469   5  0      0  375.826051  370.011274  0.05  0.51  0.58  0.65   
6   73.629285   6  0      0  367.781624  359.666144  0.06  0.52  0.59  0.64   
7   73.121660   7  0      0  355.103580  347.900428  0.06  0.52  0.61  0.63   
8   74.455365   8  0      0  348.252904  341.881058  0.06  0.51  0.60  0.62   
9   72.891773   9  0      0  344.271043  337.732601  0.07  0.50  0.59  0.57   
10  73.611951  10  0      0  342.455144  334.467778  0.08  0.49  0.58  0.49   
11  69.826844  11  0      0  341.886738  331.887525  0.08  0.49  0.57  0.46   

          
   OTHER  
0   0.84  
1   0.78  
2   0.67  
3   0.57  
4   0.45  
5   0.50  
6   0.47  
7   0.41  
8   0.40  
9   0.38  
10  0.37  
11  0.37

 0 31943 303784.0 ! ; nice & bought love price perfect quality recommend
   0-1 7712 55262.0 cover ! color keyboard mac love pro bottom apple hard
     0-1-1 265 619.0 bottom part top piece place portion snapped feet properly lid
     0-1-2 7441 8169.0 bottom protector screen speck mbp top feet finish keyboard chromebook
     0-1-3 6 2.0 big wonderful zips finding flimsy flexible flaw flat flash flap
   0-2 7732 55354.0 sleeve pocket power netbook mouse room protection ipad cord charger
     0-2-1 7523 8946.0 camera compartment sleeve room inch accessories lens plenty phone pockets
     0-2-2 209 268.0 port cable install plug dvd unit installed base hole usb
   0-3 4589 38634.0 carry pockets shoulder strap compartments comfortable straps books handle work
     0-3-1 3621 6553.0 camera compartment lenses pack accessories bottle pockets gear lens water
     0-3-2 785 784.0 son lasted important binders loves compact bought heavy double owned
     0-3-3 183 173.0 comments solution folders 